In [ ]:
import numpy as np
import math
from math import erfc

In [ ]:
"""This section reads a geometry"""

def vol(alpha,beta,gamma):

    """ Calculate volume """

    v = math.sqrt(1-math.cos(alpha)**2-math.cos(beta)**2-math.cos(gamma)**2+2*math.cos(alpha)*math.cos(beta)*math.cos(gamma))
    return v
    
def red2cart(xr,yr,zr,aa,bb,cc,alpha,beta,gamma):

    """ Convert fractional coordinates to cartesian"""

    vv = vol(alpha,beta,gamma)
    x = aa*xr+bb*math.cos(gamma)*yr+cc*math.cos(beta)*zr
    y = bb*math.sin(gamma)*yr+zr*cc*(math.cos(alpha)-math.cos(beta)*math.cos(gamma))/math.sin(gamma)
    z = zr*cc*vv/math.sin(gamma)

    return x, y, z

def ang(x, y):
    
    """ Computes angle between 2 vectors """
    
    return math.acos(np.dot(x,y)/np.dot(x,x))
    
def read_poscar(filename):
    
    """ Read POSCAR format """
    
    fin = open(filename,"r")
    r = fin.readlines()
    scale_pos = float(r[1])
    cell0 = []
    for i in range(3):
        for j in range(3):
            cell0.append(float(r[2+i].split()[j])*scale_pos)
    ntypes = int(len(r[5].split()))

    t,nat = [],[]
    for i in range(ntypes):
        t.append(r[5].split()[i])
    for i in range(len(r[6].split())):
        nat.append(int(r[6].split()[i]))

    a = [cell0[0],cell0[1],cell0[2]]
    b = [cell0[3],cell0[4],cell0[5]]
    c = [cell0[6],cell0[7],cell0[8]]
    aa = math.sqrt(np.dot(a,a))
    bb = math.sqrt(np.dot(b,b))
    cc = math.sqrt(np.dot(c,c))
    alpha = ang(b,c)
    beta = ang(c,a)
    gamma = ang(a,b)
    cell = [aa,bb,cc,alpha,beta,gamma]

    if r[7].split()[0] == "Selective":
        start = 9
    else:
        start = 8

    ctype = r[start-1].split()[0]

    if ctype == "Direct":
        xr,yr,zr,x,y,z=[],[],[],[],[],[]
        for i in range(start,start+sum(nat)):
            xr.append(float(r[i].split()[0]))
            yr.append(float(r[i].split()[1]))
            zr.append(float(r[i].split()[2]))
        for i in range(len(xr)):
            xx,yy,zz = red2cart(xr[i],yr[i],zr[i],aa,bb,cc,alpha,beta,gamma)
            x.append(xx)
            y.append(yy)
            z.append(zz)

    if ctype == "Cartesian":
        x,y,z=[],[],[]
        for i in range(start,len(r)):
            x.append(float(r[i].split()[0]))
            y.append(float(r[i].split()[1]))
            z.append(float(r[i].split()[2]))

        fin.close()
    return nat,t,x,y,z,cell0,cell,ctype

In [ ]:
nat_list,t,x,y,z,cell0,cell,ctype = read_poscar("SiC_Nico_cart")

In [ ]:
def EEqeq(x,y,z,t,nat,a1,a2,a3):
    
    """ This compute EQEq according to Ref. [] """

    Nmax =2
    K = 14.4
    zeta = 0.27                           # denotes the Ewald splitting parameter, choice of it is made based on your system
    lmda = 1.2
    chi = {"Si":1.8038,"C":5.7254}       # denotes the electronegativity in eV
    eta = {"Si":14.7704,"C":13.847}   # Eta denotes the self-Coulomb potential in eV
    gamma = {"Si":0.8218,"C":0.8712}     # denotes shielded Coulomb constant in distance units. 
    
    """Calculation of the K-space vectors"""
    vf = 2*np.pi/np.dot(a1,np.cross(a2,a3))
    b1 = 2*np.pi*np.cross(a2,a3)/np.dot(a1,np.cross(a2,a3))
    b2 = 2*np.pi*np.cross(a3,a1)/np.dot(a2,np.cross(a3,a1))
    b3 = 2*np.pi*np.cross(a1,a2)/np.dot(a3,np.cross(a1,a2))
    print(b1,b2,b3)

    def JEEQeq(i,j,x,y,z,t,nat,a1,a2,a3): 

        if i==j: # Looking at one atom interactin with itslef
            Astar = 0
            Bstar = 0
            for u in range(-Nmax,Nmax+1):
                for v in range(-Nmax,Nmax+1):
                    for w in range(-Nmax,Nmax+1):
                        if not (u==0 and v==0 and w==0): # Interacting with its periodic images
                            dx = u*a1[0]+v*a2[0]+w*a3[0]
                            dy = u*a1[1]+v*a2[1]+w*a3[1]
                            dz = u*a1[2]+v*a2[2]+w*a3[2]
                            dh1 = u*b1[0]+v*b2[0]+w*b3[0]   
                            dh2 = u*b1[1]+v*b2[1]+w*b3[1]
                            dh3 = u*b1[2]+v*b2[2]+w*b3[2]
                            dr = np.sqrt(dx**2+dy**2+dz**2)
                            dh = np.sqrt(dh1**2+dh2**2+dh3**2)
                            Astar += erfc(dr*np.sqrt(zeta))*1./dr
                            Bstar += (2*vf/dh**2)*np.exp(-(dh*0.5)**2/zeta)
            return eta[t[i]]+0.5*lmda*K*(Astar+Bstar-2*np.sqrt(zeta/np.pi))
        
        else: # Looking at one atom interacting with others
            Alpha = 0
            Beta = 0
            for u in range(-Nmax,Nmax+1):
                for v in range(-Nmax,Nmax+1):
                    for w in range(-Nmax,Nmax+1): # Interacting with any other atom in any other cell
                        xi,yi,zi = x[i],y[i],z[i]
                        xj,yj,zj = x[j],y[j],z[j]
                        dx = xj-xi+u*a1[0]+v*a2[0]+w*a3[0]
                        dy = yj-yi+u*a1[1]+v*a2[1]+w*a3[1]
                        dz = zj-zi+u*a1[2]+v*a2[2]+w*a3[2]
                        dr = np.sqrt(dx**2+dy**2+dz**2)
                        Alpha += erfc(dr*np.sqrt(zeta))*1/dr
                        if not (u==0 and v==0 and w==0): # Interacting with other atoms in other cell than itself
                            dh1 = u*b1[0]+v*b2[0]+w*b3[0]   
                            dh2 = u*b1[1]+v*b2[1]+w*b3[1]
                            dh3 = u*b1[2]+v*b2[2]+w*b3[2]
                            xi,yi,zi = x[i],y[i],z[i]
                            xj,yj,zj = x[j],y[j],z[j]
                            dx = xj-xi 
                            dy = yj-yi 
                            dz = zj-zi
                            dh = np.sqrt(dh1**2+dh2**2+dh3**2)
                            Beta += (2*vf/dh**2)*np.exp(-(dh*0.5)**2/zeta)*np.cos(dx*dh1+dy*dh2+dz*dh3)
            return 0.5*lmda*K*(Alpha+Beta)
    Qtot = 0.0    
    b = np.array([0.0]*nat)
    b[0] = Qtot
    A = np.matrix([[0.0]*nat]*nat)
    A[0] = [1]*nat  # First row = 1
    for i in range(1,nat):
        b[i] = chi[t[i]]-chi[t[0]] 
        for j in range(nat):
            A[i,j] = JEEQeq(i,j,x,y,z,t,nat,a1,a2,a3)-JEEQeq(0,j,x,y,z,t,nat,a1,a2,a3)
        print(i,nat)
            
    #print("This is the A matrix")
    #print(A)
    #print("This is the b vector")
    #print(b)
    Q = np.linalg.solve(A, -b)
    
    print("This is the partial charge of each atom type")
    for i in range(nat):
        print(t[i],Q[i])
 

In [ ]:
type_list = []
for i in range(len(t)):
    type_list += [t[i]]*nat_list[i]

#print(type_list)
a1 = cell0[:3]
a2 = cell0[3:6]
a3 = cell0[6:]
nat = sum(nat_list)

#print(nat,type_list,a1,a2,a3)
EEqeq(x,y,z,type_list,nat,a1,a2,a3)